# Tema 24: Tamaños de muestra

En seguimiento a la libreta previa donde hablamos sobre los tamaños de muestra, esta libreta mostrará cómo realizar el cálculo del tamaño muestral en varios escenarios.


:::{important}

Esta libreta se divide en dos secciones. En la primera utilizaremos las fórmulas más comunes para el cálculo de tamaños muestrales, y utilizaremos `scipy` y `numpy`, que ya debes tener instalados. En la segunda parte, utilizaremos la librería `statsmodels` que también ya deberías tener disponible.

:::


## Librerías

In [ ]:
from math import ceil # redondea hacia arriba

import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
print('Librerías importadas')

Librerías importadas


## Estimación para proporciones
### Fórmula de Cochran

La primera fórmula que revisaremos es la **fórmula de Cochran**, utilizada para estimar el tamaño de muestra necesario al estudiar una proporción en un estudio descriptivo.

$$
n = \frac{(Z_{1-\alpha/2})^2 \cdot p \cdot (1-p)}{d^2}
$$

Donde:

- $Z_{1-\alpha/2}$ es el valor crítico de la distribución normal estándar para un nivel de confianza dado, en este caso para el error de tipo 1.
- $p$ es la proporción esperada,
- $d$ es el margen de error (precisión deseada).

In [21]:
def cochran_una_p(p, d, alfa=0.05, hipotesis='dos colas'):
    """
    Calcula el tamaño de muestra para una proporción usando la fórmula de Cochran.

    Parámetros:
    - p: proporción esperada (entre 0 y 1)
    - d: margen de error (precisión)
    - alfa: nivel de significancia (por defecto 0.05)
    - hipotesis: 'dos colas' o 'una cola'

    Retorna:
    - n: tamaño de muestra (entero)
    """
    hips = 'dos colas', 'una cola'
    if hipotesis not in hips:
        raise ValueError("La hipótesis debe ser 'dos colas' o 'una cola'.")

    if hipotesis == 'dos colas':
        z = stats.norm.ppf(1-alfa/2)
    else:
        z = stats.norm.ppf(1-alfa)
        
    n = (z**2 * p * (1 - p)) / d**2
    return ceil(n)
print('Función definida')

Función definida


Probemos la función con los siguientes parámetros:
- Nivel de significancia: $\alpha = 0.05$
- Proporción esperada: $p = 0.5$
- Margen de error: $d = 0.1$
- Hipótesis: una cola

In [19]:
cochran_una_p(0.5, 0.1, hipotesis='una cola')

68

Para este estudio, requeriríamos de al menos 68 pacientes.

### Estudios de equivalencia en una proporción

En este caso, queremos conocer si la proporción estimada es diferente de un valor preestablecido, para ello, se modifica ligeramente la fórmula para incluir el poder del estudio.

$$
n = \frac{(Z_{1-\alpha/2} + Z_\beta)^2 \cdot p \cdot (1-p)}{d^2}
$$

Donde:
- $Z_\beta$ es el valor crítico de la distribución normal estándar para un nivel de confianza dado, en este caso para el error de tipo 2.


In [27]:


def cochran_eq_una_p(p, d, alfa=0.05, beta=0.2, hipotesis='dos colas'):
    """
    Calcula el tamaño de muestra para detectar diferencia en una proporción,
    incorporando el poder estadístico (estudios de equivalencia o hipótesis).

    Parámetros:
    - p: proporción esperada (entre 0 y 1)
    - d: diferencia mínima que se desea detectar (margen de error)
    - alfa: nivel de significancia (por defecto 0.05)
    - beta: error tipo II (por defecto 0.2, que equivale a 80% de poder)

    Retorna:
    - n: tamaño de muestra (entero)
    """
    hips = 'dos colas', 'una cola'
    if hipotesis not in hips:
        raise ValueError("La hipótesis debe ser 'dos colas' o 'una cola'.")

    if hipotesis == 'dos colas':
        z_alfa = stats.norm.ppf(1-alfa/2)
    else:
        z_alfa = stats.norm.ppf(1-alfa)

    z_beta = stats.norm.ppf(1 - beta)
    n = ((z_alfa + z_beta)**2 * p * (1 - p)) / d**2
    return ceil(n)

print('Función definida')

Función definida


Probemos la función con los siguientes parámetros:
- Nivel de significancia: $\alpha = 0.05$
- Probabilidad de error de tipo 2: $\beta = 0.1$
- Proporción esperada: $p = 0.3$
- Margen de error: $d = 0.1$
- Hipótesis: una cola

In [25]:
cochran_eq_una_p(0.3, 0.1, beta=0.2, hipotesis='una cola')

130

Para este estudio requeriríamos entonces 130 sujetos.

### Una proporción finita (Cochran)

Existen situaciones en las que queremos acotar el tamaño muestral a una población conocida, para lo cual se realiza un ajuste a en la fórmula de Cochran.


def una_prop_fin(p, error, N):
  z = 1.96**2
  q = 1-p
  num = N*z*p*q
  den = error**2*(N-1)+z*p*q
  n = num/den
  return n




def una_media(s, error):
  z = 1.96
  n = (z*s/error)**2
  return n


def un_odds_ratio(
  OR: float,
  p0: float,
  alpha: float=0.05,
  beta: float=0.2,
  ratio: int=1
) -> tuple[int, int]:
    """
    Calcula el tamaño de muestra para un odds ratio
    Args:
      OR: El Odds ratio a estimar
      p0: prevalencia general
      alpha: error tipo 1
      beta: error tipo 2
      ratio: razón de controles/casos
    Returns:
      int, int: n_casos,n_controles
    """
    power = 1-beta

    # p1 (prevalencia de exposición en casos)
    p1 = (OR * p0) / (1 + p0 * (OR - 1))
    # Z-scores a dos colas
    Za = stats.norm.ppf(1 - alpha/2).round(2) # e.j. ~1.96 para alpha=0.05
    Zb = stats.norm.ppf(power).round(2)  # e.j. ~0.84 para 80% power
    # Pooled exposure proportion
    p_bar = (p1 + ratio * p0) / (1 + ratio)
    q_bar = 1 - p_bar

    # Calculatar
    n_cases = (
        Za * math.sqrt((1 + ratio) * p_bar * q_bar)
      + Zb * math.sqrt(p0 * (1 - p0) + ratio * p1 * (1 - p1))
    )**2 / (ratio * (p1 - p0)**2)
    n_controls = ratio * n_cases
    # Round up
    return math.ceil(n_cases), math.ceil(n_controls)

def for_chi2_indep(p_ij, alpha=0.05, beta=0.2):
    """
    Compute sample size for chi-square test of independence in r x c table.
    Always uses brentq method.

    Parameters:
    p_ij (2D array): Expected joint probabilities matrix (r x c), must sum to 1.
    alpha (float): Type I error (e.g., 0.05)
    beta (float): Type II error (e.g., 0.2 for 80% power)

    Returns:
      n (float): Required sample size

    Add:
      Taken from Chow 2008 6.2.1
    """

    # 1. Check that p_ij is valid
    if not np.isclose(np.sum(p_ij), 1):
        raise ValueError("The matrix p_ij must sum to 1.")

    r = len(p_ij)
    c = len(p_ij[0] if r else 0)

    # 2. Degrees of freedom
    df = (r - 1) * (c - 1)

    # 3. Critical value under central chi-squared
    critical_value = stats.chi2.ppf(1 - alpha, df)

    # 4. Objective function for root finding
    def objective(delta):
        return stats.ncx2.sf(critical_value, df, delta) - (1 - beta)

    # 5. Solve for delta using brentq
    sol = root_scalar(objective, bracket=[0, 50], method='brentq')
    if not sol.converged:
        raise RuntimeError("Root finding did not converge.")

    delta = sol.root

    # 6. Marginals
    p_i_dot = np.sum(p_ij, axis=1)  # row sums
    p_dot_j = np.sum(p_ij, axis=0)  # column sums

    # 7. Compute the noncentrality effect structure
    effect_size = 0.0
    for i in range(r):
        for j in range(c):
            expected = p_i_dot[i] * p_dot_j[j]
            if expected > 0:
                effect_size += ((p_ij[i, j] - expected) ** 2) / expected

    # 8. Sample size
    n = delta / effect_size

    return n


def dos_props(p1, p2):
    num = p1*(1-p1) + p2 * (1-p2)
    den = (p1-p2)**2
    z = (1.96 + 0.84)**2
    return z * num/den

def dos_medias(mu1, mu2, s):
  d = (mu1-mu2)/s
  za = 1.96
  zb = 0.84
  n = (
      (2 * (za+zb)**2)
      / d**2
  ) + 0.25 * za**2
  return math.ceil(n)
